# Predicción de supervivéncia del Titanic

Esta práctica se divide en dos partes:
1. En la primera parte entrenaremos un conjunto de modelos de clasificación de machine learning (perceptron, regression logística y arbol de decisión) para predecir la probabilidad que tendría un pasajero del Titánic de sobrevivir.
2. En la segunda parte realizaremos el *feature importance*, que básicamente consiste en analizar la importancia que tiene cada característica en cada uno de los modelos predictivos

![OHE](https://imgs.search.brave.com/Cn_B5VSZhmV_ElcnW0wlfHZ4rn5MCPijlzLBp2IA5Eo/rs:fit:844:225:1/g:ce/aHR0cHM6Ly90c2Uy/Lm1tLmJpbmcubmV0/L3RoP2lkPU9JUC5y/RGgzRWs0dEl3QXZD/eVQ2RV9ENVpBSGFF/SyZwaWQ9QXBp)


## Parte 1

### 1.1 Importar librerias
Importamos la librerias que usaremos

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

import pandas as pd
import numpy as np

### 1.2 Cargamos el dataset
Cargamos el dataset para poder visualizar los datos que usaremos.

In [4]:
df = pd.read_csv("./dades.csv")

Para poder ver todas las columnas usamos la sigüiente instrucción de *pandas*

In [8]:
pd.set_option('display.max_columns',None)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 1.3 Limpieza de datos
Para poder entrenar a nuestros modelos previamente debemos "*masticar*" los datos